## Integração com APIs: Feriados e Tempo

### Utilize as APIs públicas abaixo para responder às questões 1-8:
- [Public Holiday API](https://date.nager.at/Api)
- [Open-Meteo Historical Weather API](https://open-meteo.com/)

1. **Quantos feriados há no Brasil em todo o ano de 2024?**

In [1]:
import requests

In [2]:
class HollidayApi:
    URL = 'https://date.nager.at'

    @classmethod
    def available_countries(cls) -> list[dict[str, str]]:
        response = requests.get(f'{cls.URL}/api/v3/AvailableCountries')
        return response.json()
    
    @classmethod
    def public_hollidays(cls, year: int, country_code: str):
        response = requests.get(
            f'{cls.URL}/api/v3/PublicHolidays/{year}/{country_code}'
        )
        return response.json()

In [3]:
countries = HollidayApi.available_countries()
brazil_code = next(
    (country["countryCode"] for country in countries if country["name"] == "Brazil"),
    None,
)
hollidays = HollidayApi.public_hollidays(year=2024, country_code=brazil_code)
len(hollidays)

14

__Resposta:__ Há 14 feriados no Brasil no ano de 2024. 

2. **Qual mês de 2024 tem o maior número de feriados?**

In [4]:
from datetime import date

from babel.dates import format_date

In [5]:
hollidays_per_month = {}
for holliday in hollidays:
    month = format_date(date.fromisoformat(holliday["date"]), "MMMM")
    hollidays_per_month[month] = hollidays_per_month.get(month, 0) + 1
max_hollidays = max(hollidays_per_month.values())
max_months = [
    month
    for month, hollidays_count in hollidays_per_month.items()
    if hollidays_count == max_hollidays
]
print(*max_months, sep="\n")

fevereiro
março
maio
novembro


__Resposta:__ Os meses de Fevereiro, Março, Maio e Novembro tem 2 feriados cada um, sendo o maior número de feriados em algum mês em 2024.

3. **Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

In [6]:
sum(
    1
    for holliday in hollidays
    if date.fromisoformat(holliday["date"]).weekday() < 5 # sáb = 5, dom = 6
)

9

__Resposta:__ Em 2024, 9 feriados caem em dias de semana.

4. **Qual foi a temperatura média em cada mês?**  
   Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024.

In [7]:
from collections import OrderedDict
from urllib.parse import quote

In [8]:
class OpenMeteoApi:
    URL = "https://archive-api.open-meteo.com/v1/archive"

    @classmethod
    def historical_temperature_and_weather(
        cls,
        latitude: float = -22.9064,
        longitude: float = -43.1822,
        start_date: date = date(2024, 1, 1),
        end_date: date = date(2024, 8, 1),
        timezone: str = "America/Sao_Paulo",
    ) -> dict[str, float | int | str | dict[str, str | list[str | int | float]]]:
        params = OrderedDict(
            [
                ("latitude", latitude),
                ("longitude", longitude),
                ("start_date", start_date.strftime(r"%Y-%m-%d")),
                ("end_date", end_date.strftime(r"%Y-%m-%d")),
                ("daily", "weather_code,temperature_2m_mean"),
                ("timezone", quote(timezone)),
            ]
        )
        query_str = "&".join(f"{key}={value}" for key, value in params.items())
        response = requests.get(f"{cls.URL}?{query_str}")
        return response.json()

    @classmethod
    def weather_codes(cls) -> dict[str, dict[str, dict[str, str]]]:
        url_weather_codes = "https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/76b0cb0ef0bfd8a2ec988aa54e30ecd1b483495d/descriptions.json"
        response = requests.get(url_weather_codes)
        return response.json()


In [9]:
response_json = OpenMeteoApi.historical_temperature_and_weather()
daily = response_json["daily"]
dates = list(map(lambda d_str: date.fromisoformat(d_str), daily["time"]))
temperatures = daily["temperature_2m_mean"]
month_tempsum = {}
for day, temp in zip(dates, temperatures):
    month = format_date(day, "MMMM", locale="pt_BR")
    month_tempsum[month] = {
        "sum": month_tempsum.get(month, {}).get("sum", 0) + temp,
        "count": month_tempsum.get(month, {}).get("count", 0) + 1,
    }
month_tempmean = {m: ts["sum"] / ts["count"] for m, ts in month_tempsum.items()}
for month, tempmean in month_tempmean.items():
    print(f"|{month}|{tempmean:.1f}|")


|janeiro|26.7|
|fevereiro|27.2|
|março|26.4|
|abril|25.2|
|maio|25.0|
|junho|22.6|
|julho|21.1|
|agosto|21.2|


Abaixo as médias para cada mês:

|Mês|Temperatura Média|
|---|---|
|janeiro|26.7|
|fevereiro|27.2|
|março|26.4|
|abril|25.2|
|maio|25.0|
|junho|22.6|
|julho|21.1|

5. **Qual foi o tempo predominante em cada mês nesse período?**  
   Utilize como referência para o código de tempo (_weather_code_) o seguinte link: [WMO Code](https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c).

In [10]:
WEATHER_TRANSLATION_KEY = {
    # fonte: http://www.sistema-alerta-rio.com.br/previsao-do-tempo/termosmet/
    # fonte: https://www.weather.gov/otx/Full_Weather_Glossary
    # precisei adicionar os termos para chuvisco
    "Light Drizzle": "Chuvisco leve",
    "Cloudy": "Céu Encoberto", #
    "Mainly Sunny": "Céu parcialmente nublado",
    "Mainly Clear": "Céu parcialmente nublado",
    "Rain": "Chuva moderada", # escolha baseada nos patamares de chuva disponíveis na API
    "Sunny": "Céu claro",
    "Clear": "Céu claro",
    "Drizzle": "Chuvisco", # escolha baseada nos patamares de chuva disponíveis na API
    "Partly Cloudy": "Céu nublado",
    "Light Rain": "Chuva fraca",
    # "": "",
}

In [11]:
weather_dict = OpenMeteoApi.weather_codes()
weathers = daily["weather_code"]
month_weatherfreq = {}
for day, weather in zip(dates[:-1], weathers):
    month = format_date(day, "MMMM", locale="pt_BR")
    month_weatherfreq[month] = month_weatherfreq.get(month, {})
    month_weatherfreq[month][weather] = month_weatherfreq[month].get(weather, 0) + 1
month_weather = {}
for month, weather_freq in month_weatherfreq.items():
    max_freq = max(weather_freq.values())
    weather_codes = [
        weather_dict[str(weather)]
        for weather, freq in weather_freq.items()
        if freq == max_freq
    ]
    month_weather[month] = {"days_count": max_freq, "weather_codes": weather_codes}
for month, count_weathers in month_weather.items():
    days_count = count_weathers["days_count"]
    weather_codes = count_weathers["weather_codes"]
    for weather in weather_codes:
        day = WEATHER_TRANSLATION_KEY.get(
            weather["day"]["description"], weather["day"]["description"]
        )
        night = WEATHER_TRANSLATION_KEY.get(
            weather["night"]["description"], weather["night"]["description"]
        )
        print(f"|{month}|{day}|{night}|{days_count}|")

|janeiro|Chuva moderada|Chuva moderada|10|
|fevereiro|Chuvisco leve|Chuvisco leve|13|
|março|Chuvisco leve|Chuvisco leve|9|
|abril|Céu Encoberto|Céu Encoberto|7|
|maio|Chuvisco leve|Chuvisco leve|10|
|junho|Céu parcialmente nublado|Céu parcialmente nublado|12|
|julho|Céu Encoberto|Céu Encoberto|8|
|julho|Chuvisco leve|Chuvisco leve|8|


Tempo predominante em cada mês e a duração:

|Mês|Tempo de Dia|Tempo de Noite|Duração (em dias)|
|---|---|---|---|
|janeiro|Chuva moderada|Chuva moderada|10|
|fevereiro|Chuvisco leve|Chuvisco leve|13|
|março|Chuvisco leve|Chuvisco leve|9|
|abril|Céu Encoberto|Céu Encoberto|7|
|maio|Chuvisco leve|Chuvisco leve|10|
|junho|Céu parcialmente nublado|Céu parcialmente nublado|12|
|julho|Céu Encoberto|Céu Encoberto|8|
|julho|Chuvisco leve|Chuvisco leve|8|


Quando houver "empate" no tempo mais frequente, todos os tempos são exibidos com a duração.

6. **Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

In [12]:
daily_weather_temp = dict(zip(dates, zip(temperatures, weathers)))
dates_set = set(dates)
holliday_info = OrderedDict()
for holliday in hollidays:
    day = date.fromisoformat(holliday["date"])
    if day not in dates_set:
        continue
    weather_code_dict = weather_dict[str(daily_weather_temp[day][1])]
    weather_day = WEATHER_TRANSLATION_KEY.get(
        weather_code_dict["day"]["description"], weather_code_dict["day"]["description"]
    )
    weather_night = WEATHER_TRANSLATION_KEY.get(
        weather_code_dict["night"]["description"],
        weather_code_dict["night"]["description"],
    )
    holliday_info[day] = {
        "name": holliday["localName"],
        "temperature": daily_weather_temp[day][0],
        "day": weather_day,
        "night": weather_night,
    }
for day, info in holliday_info.items():
    name = info["name"]
    temp = info["temperature"]
    weather_day = info["day"]
    weather_night = info["night"]
    print(f"|{day:%d/%m/%Y}|{name}|{temp}|{weather_day}|{weather_night}|")

|01/01/2024|Confraternização Universal|24.9|Chuvisco leve|Chuvisco leve|
|12/02/2024|Carnaval|30.2|Céu parcialmente nublado|Céu parcialmente nublado|
|13/02/2024|Carnaval|30.5|Chuvisco leve|Chuvisco leve|
|29/03/2024|Sexta-feira Santa|25.0|Chuva fraca|Chuva fraca|
|31/03/2024|Domingo de Páscoa|24.5|Chuvisco|Chuvisco|
|21/04/2024|Dia de Tiradentes|23.1|Céu nublado|Céu nublado|
|01/05/2024|Dia do Trabalhador|28.1|Céu claro|Céu claro|
|30/05/2024|Corpus Christi|20.8|Chuvisco leve|Chuvisco leve|
|09/07/2024|Revolução Constitucionalista de 1932|21.5|Chuvisco|Chuvisco|


Tempo e temperatura média em cada feriado:

|Data|Feriado|Temperatura|Tempo durante o dia|Tempo durante a noite|
|---|---|---|---|---|
|01/01/2024|Confraternização Universal|24.9|Chuvisco leve|Chuvisco leve|
|12/02/2024|Carnaval|30.2|Céu parcialmente nublado|Céu parcialmente nublado|
|13/02/2024|Carnaval|30.5|Chuvisco leve|Chuvisco leve|
|29/03/2024|Sexta-feira Santa|25.0|Chuva fraca|Chuva fraca|
|31/03/2024|Domingo de Páscoa|24.5|Chuvisco|Chuvisco|
|21/04/2024|Dia de Tiradentes|23.1|Céu nublado|Céu nublado|
|01/05/2024|Dia do Trabalhador|28.1|Céu claro|Céu claro|
|30/05/2024|Corpus Christi|20.8|Chuvisco leve|Chuvisco leve|
|~~09/07/2024~~|~~Revolução Constitucionalista de 1932~~|~~21.5~~|~~Chuvisco~~|~~Chuvisco~~|

Observação: dia 9 de julho não é feriado no Rio de Janeiro.

7. **Considere as seguintes suposições:**
   - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
   - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
   - O cidadão carioca só vai à praia quando não está com frio;
   - O cidadão carioca também só vai à praia em dias com sol, evitando dias **totalmente** nublados ou chuvosos (considere _weather_code_ para determinar as condições climáticas).

   Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

In [13]:
NO_BEACH_WEATHERS = {
    "Chuvisco leve",
    "Chuvisco",
    "Chuva fraca",
    "Chuva moderada",
    "Céu Encoberto", 
    "Céu nublado",
}

In [14]:
for day, info in holliday_info.items():
    if (
        info["name"] == "Revolução Constitucionalista de 1932"
        or info["temperature"] > 20
        and info["day"] not in NO_BEACH_WEATHERS
        # or info["night"] in NO_BEACH_WEATHERS
    ):
        continue
    name = info["name"]
    temp = info["temperature"]
    weather_day = info["day"]
    # weather_night = info["night"]
    print(f"|{day:%d/%m/%Y}|{name}|{temp}|*{weather_day}*|")

|01/01/2024|Confraternização Universal|24.9|*Chuvisco leve*|
|13/02/2024|Carnaval|30.5|*Chuvisco leve*|
|29/03/2024|Sexta-feira Santa|25.0|*Chuva fraca*|
|31/03/2024|Domingo de Páscoa|24.5|*Chuvisco*|
|21/04/2024|Dia de Tiradentes|23.1|*Céu nublado*|
|30/05/2024|Corpus Christi|20.8|*Chuvisco leve*|


Feriados não aproveitáveis:

|Data|Feriado|Temperatura|Tempo durante o dia|
|---|---|---|---|
|01/01/2024|Confraternização Universal|24.9|*Chuvisco leve*|
|13/02/2024|Carnaval|30.5|*Chuvisco leve*|
|29/03/2024|Sexta-feira Santa|25.0|*Chuva fraca*|
|31/03/2024|Domingo de Páscoa|24.5|*Chuvisco*|
|21/04/2024|Dia de Tiradentes|23.1|*Céu nublado*|
|30/05/2024|Corpus Christi|20.8|*Chuvisco leve*|

**Nota**: em todos a temperatura foi maior que 20ºC, então a fonte de não aproveitamento foi o tempo; usei apenas o dia porque é quando a maior parte das pessoas vai à praia

8. **Qual foi o feriado "mais aproveitável" de 2024?**  
   Considere o melhor par tempo e temperatura.

In [15]:
BEACH_WEATHERS_SCORE = {
    "Céu nublado": 1/3,
    "Céu parcialmente nublado": 2/3,
    "Céu claro": 1,
}

In [16]:
beach_holliday_info = []
for day, info in holliday_info.items():
    if (
        info["name"] == "Revolução Constitucionalista de 1932"
        or info["temperature"] < 20
        or info["day"] not in BEACH_WEATHERS_SCORE.keys()
        # or info["night"] in NO_BEACH_WEATHERS
    ):
        continue
    name = info["name"]
    temp = info["temperature"]
    weather_day = info["day"]
    beach_holliday_info.append(
        {"date": day, "name": name, "temperature": temp, "day": weather_day}
    )
t1 = max(h["temperature"] for h in beach_holliday_info)
t0 = min(h["temperature"] for h in beach_holliday_info)
for idx, holliday_info in enumerate(beach_holliday_info):
    t = beach_holliday_info[idx]["temperature"]
    score_t = (t - t0) / (t1 - t0)
    score_w = BEACH_WEATHERS_SCORE[holliday_info["day"]]
    beach_holliday_info[idx]["score"] = (score_t + score_w) / 2
best_holliday = max(beach_holliday_info, key=lambda h: h["score"])
print(best_holliday, sep="\n")

{'date': datetime.date(2024, 5, 1), 'name': 'Dia do Trabalhador', 'temperature': 28.1, 'day': 'Céu claro', 'score': 0.8521126760563381}


O feriado mais aproveitável de 2024 foi o dia do trabalhador (01/05/2024) que teve o melhor equilíbrio entre tempo e temperatura. Num critério alternativo, o dia do trabalhador foi o único feriado totalmente ensolarado, portanto o melhor também assim.